# Notion

This notebook is the modified notebook of [AtomisticSimulationTutorial](https://docs.matlantis.com/atomistic-simulation-tutorial/en/) to run on Google Colab. We only provide executable parts with general library.

# [Advanced] Nanoparticle energy

This section contains advanced content. Those who wish to hurry to the next chapter may skip it and read it later.

In this section, we will try to calculate the excess energy of nanoparticles (cluster structures) and create adsorption structures on supports.

## Initial setup

In [1]:
!pip install ase

Looking in indexes: https://pypi.org/simple, http://pypi.artifact.svc:8080/simple

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

from IPython.display import HTML, Image

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation

from ase import Atoms
from ase.build import make_supercell
from ase.io import write
from ase.visualize import view

os.makedirs("output", exist_ok=True)

def view_x3d(atoms, idx=0):
    if isinstance(atoms[0], Atoms):
        # Assume this is a trajectory or struct list
        if (len(atoms) <= idx):
                print(f"The specified index exceeds the length of the trajectory. The length of the trajectory is {len(atoms)}.")
        return view(atoms[idx], viewer="x3d")
    else:
        return view(atoms, viewer="x3d")


def view_ase_atoms(atoms, rotation="0x,0y,0z", figsize=(4, 4), title="", scale=100):
    fig, ax = plt.subplots(figsize=figsize)
    write("output/tmp.png", atoms, rotation=rotation, scale=scale)
    img = mpimg.imread('output/tmp.png')
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')
    plt.show()
    os.remove('output/tmp.png')
    return

def traj_to_apng(traj, rotation='30x,30y,30z'):
    imgs = []
    for atom in traj:
        supercell = make_supercell(atom, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        write('output/tmp.png', supercell, rotation=rotation, show_unit_cell=2)
        img = mpimg.imread('output/tmp.png')
        imgs.append(img)
    os.remove('output/tmp.png')

    fig, ax = plt.subplots()

    def update(frame):
        img = imgs[frame]
        ax.clear()
        ax.imshow(img)
        return []

    ani = FuncAnimation(fig, update, frames=len(imgs), blit=True)
    plt.close()
    return HTML(ani.to_jshtml())

In [3]:
from ase.calculators.emt import EMT

calculator = EMT()

In [4]:
from ase.cluster import Decahedron, Icosahedron, Octahedron, wulff_construction
from ase import Atoms 
from ase.build import bulk
from ase.constraints import ExpCellFilter, StrainFilter
from ase.optimize import LBFGS
from ase.io import Trajectory
import numpy as np
import pandas as pd

## Excess energy

A material that is a pure metal mixed with one or more other elements is called an **alloy**.
The property of metals changes significantly when combining two or more elements.

For example, [duralumin](https://en.wikipedia.org/wiki/Duralumin), a mixture of aluminum and copper or magnesium, is used in aerospace equipment because of its lightweight and hardness.

A mixture of two elements is called a binary alloy, while a mixture of three or more elements is called a multinary alloy.

Multinary alloys are expected to lead to the discovery of materials with new properties.
As the types of elements increase, the combinations of their composition ratios increase, 
and this is an area that has not yet been analyzed. <br/>
[Matlantis](https://matlantis.com) can handle such systems, taking advantage of its versatility.

When considering the atomic structure of these alloys, it is necessary to know how they mix in reality and whether they are combinations that can be mixed in the first place.

Excess energy is a measure of the stability of such alloys. <br/>
The excess energy of a binary alloy of element A and element B is calculated as follows and indicates how stable the energy of the alloy is compared to the case where elements A and B are not mixed together.

$$ E_{\rm{excess}} = \frac{1}{N_{\rm{alloy}}} \left( E_{\rm{alloy}} - \frac{N_{\rm{alloyA}}}{N_{\rm{alloy}}} E_{\rm{A}} - \frac{N_{\rm{alloyB}}}{N_{\rm{alloy}}} E_{\rm{B}} \right) $$

$E_{\rm{alloy}}$ is the energy of the alloy, $E_\rm{A}, E_\rm{B}$ is the energy when composed of single elements A or B, and $N_{\rm{alloy}}, N_{\rm{alloyA}}, N_{\rm{alloyB}}$ are the total number of atoms in the alloy, the number of element A atoms, and the number of element B atoms, respectively.

The same definition can be used for the multinary alloy.

Let us evaluate its stability by evaluating the excess energy for a mixed Pt and Pd structure.

Reference paper:

 - [Electronic Structure and Phase Stability of PdPt Nanoparticles](https://pubs.acs.org/doi/10.1021/acs.jpclett.5b02753)
 - [Electronic structure and phase stability of Pt3M (M = Co, Ni, and Cu) bimetallic nanoparticles](https://www.sciencedirect.com/science/article/abs/pii/S0927025620303657)
 - [Calculations of Real-System Nanoparticles Using Universal Neural Network Potential PFP](https://arxiv.org/abs/2107.00963)

We prepare Pt711 nanoparticle, and mix it with Pd.
Even when considering binary nanoparticles, there are various possibilities of how they are mixed.
This tutorial will create the following structure and evaluate its excess energy.

- Structure in which Pt and Pd are evenly mixed
- Structure with Pt in the inner shell and Pd in the outer shell
- Structure with Pd in the inner shell and Pt in the outer shell

This is a method definition to create a core shell structure. Feel free to skip the details of this method.

In [5]:
from typing import List, Tuple

from ase import Atoms, neighborlist
from ase.cluster import Cluster
import numpy as np
from ase.data import atomic_numbers


def cluster2atoms(cluster: Cluster) -> Atoms:
    """Convert ASE Cluster to ASE Atoms

    Args:
        cluster (Cluster): input cluster instance

    Returns:
        atoms (Atoms): converted output, atoms instance
    """
    return Atoms(cluster.symbols, cluster.positions, cell=cluster.cell)


def calc_coordination_numbers(atoms: Atoms) -> Tuple[List[int], List[np.ndarray]]:
    """Calculates coordination number

    Args:
        atoms: input atoms

    Returns:
        cns (list): coordination number for each atom
        bonds (list): bond destination indices for each atom
    """
    nl = neighborlist.NeighborList(
        neighborlist.natural_cutoffs(atoms), self_interaction=False, bothways=True
    )
    nl.update(atoms)
    bonds = []
    cns = []
    for i, _ in enumerate(atoms):
        indices, offsets = nl.get_neighbors(i)
        bonds.append(indices)
        cns.append(len(indices))
    return cns, bonds


def substitute(scaffold: Atoms, sites: np.ndarray, elements: List[str]) -> Atoms:
    """Substibute `elements` to `sites` indices of `scaffold` atoms

    Args:
        scaffold (Atoms): Original input atoms
        sites (np.ndarray): site indices to substitute `elements`
        elements (list): elements to be substituted

    Returns:
        substituted (Atoms): substituted ase atoms
    """
    substituted = scaffold.copy()
    for site, element in zip(sites, elements):
        substituted.numbers[site] = atomic_numbers[element]
    return substituted


def make_core_shell(scaffold: Atoms, element: str) -> Atoms:
    """Make core shell structure

    Input `scaffold` element is kept inside core shell,
    and outside surface is replaced by `element`.

    Note that it is not fully tested.
    It was not guaranteed to work on all the cases.

    Args:
        scaffold (Atoms): input cluster structure
        element (str): outside surface element to be replaced

    Returns:
        coreshell (Atoms): core shell structure
    """
    # CN : vertex < edge < surface < core
    cn = np.array(calc_coordination_numbers(scaffold)[0])
    cn_set = np.unique(cn)
    vertexes = None
    surfaces = None
    cores = None
    if len(cn_set) == 1:
        pass
    elif len(cn_set) == 2:
        cores = np.where(cn == cn_set[1])[0]
        surfaces = np.where(cn == cn_set[0])[0]
    elif len(cn_set) == 3 or len(cn_set) == 4:
        cores = np.where(cn == cn_set[-1])[0]
        surfaces = np.where(cn != cn_set[-1])[0]
        vertexes = np.where(cn == cn_set[0])[0]
    else:
        cores = np.where(cn == cn_set[-1])[0]
        surfaces = np.where(cn != cn_set[-1])[0]
        vertexes = np.where(np.isin(cn, cn_set[:-3]))[0]

    if surfaces is None:
        core_shell = scaffold.copy()
    else:
        core_shell = substitute(scaffold, surfaces, [element] * len(surfaces))
    return core_shell


First, a nanoparticle skeleton structure (scaffold) is created as a base structure.

In [6]:
Pt711 = cluster2atoms(Octahedron("Pt", 11, cutoff=4))
view_x3d(Pt711)

Create a Pd306Pt405 structure in which only the inner shell of the base skeleton is replaced with "Pd" element using the `make_core_shell` function. 

In [7]:
Pd306Pt405 = make_core_shell(Pt711, "Pd")
view_x3d(Pd306Pt405)

Similarly, create the Pd405Pt306 structure whose inner shell is Pt and outer shell is Pd.

In [8]:
Pd711 = Octahedron("Pd", 11, cutoff=4)
Pd405Pt306 = make_core_shell(Pd711, "Pt")
view_x3d(Pd405Pt306)

In [9]:
symbols, counts = np.unique(Pd405Pt306.symbols, return_counts=True)
print(f"Pd405Pt306 structure contains {symbols} with counts {counts}", )

symbols, counts = np.unique(Pd306Pt405.symbols, return_counts=True)
print(f"Pd306Pt405 structure contains {symbols} with counts {counts}", )

Pd405Pt306 structure contains ['Pd' 'Pt'] with counts [405 306]
Pd306Pt405 structure contains ['Pd' 'Pt'] with counts [306 405]


Let's apply structural relaxation for each structure to obtain its energy.

In [10]:
from ase.optimize import FIRE

def get_opt_energy(atoms, fmax=0.001):    
    opt = FIRE(atoms)
    opt.run(fmax=fmax)
    return atoms.get_total_energy()

In [11]:
print("Optimizing Pt711")
Pt711.calc = calculator
E_pt711 = get_opt_energy(Pt711)

print("Optimizing Pd306Pt405")
Pd306Pt405.calc = calculator
E_pd306pt405 = get_opt_energy(Pd306Pt405)

print("Optimizing Pd405Pt306")
Pd405Pt306.calc = calculator
E_pd405pt306 = get_opt_energy(Pd405Pt306)

print("Optimizing Pd711")
Pd711.calc = calculator
E_pd711 = get_opt_energy(Pd711)

Optimizing Pt711
      Step     Time          Energy          fmax
FIRE:    0 02:36:10      142.220129        1.903551
FIRE:    1 02:36:10      139.022863        1.711908
FIRE:    2 02:36:10      136.031090        1.476557
FIRE:    3 02:36:11      133.576951        1.210718
FIRE:    4 02:36:11      131.699618        0.913577
FIRE:    5 02:36:11      130.428044        0.587522
FIRE:    6 02:36:11      129.759662        0.469178
FIRE:    7 02:36:11      129.612373        0.728565
FIRE:    8 02:36:11      129.543677        0.697565
FIRE:    9 02:36:11      129.416959        0.638027
FIRE:   10 02:36:12      129.250882        0.554642
FIRE:   11 02:36:12      129.067406        0.453876
FIRE:   12 02:36:12      128.886343        0.343533
FIRE:   13 02:36:12      128.720446        0.261763
FIRE:   14 02:36:12      128.572553        0.320076
FIRE:   15 02:36:12      128.422639        0.361302
FIRE:   16 02:36:12      128.259064        0.374778
FIRE:   17 02:36:13      128.067231        0.3511

We can now calculate excess energy from these energies.

In [12]:
E_exess_pd306pt405 = (E_pd306pt405 - 306 / 711 * E_pd711 - 405 / 711 * E_pt711) / 711
E_exess_pd405pt306 = (E_pd405pt306 - 405 / 711 * E_pd711 - 306 / 711 * E_pt711) / 711

print(f"Excess energy of Pd306Pt405 = {E_exess_pd306pt405 * 1000:.2f} meV")
print(f"Excess energy of Pd405Pt306 = {E_exess_pd405pt306 * 1000:.2f} meV")

Excess energy of Pd306Pt405 = -20.31 meV
Excess energy of Pd405Pt306 = 14.15 meV


It suggests that Pd306Pt405, which has a negative Excess energy, is more stable when mixed than when Pd and Pt nanoparticles exist alone. <br/>
On the other hand, the Pd405Pt306 structure is more unstable than Pd306Pt405 since its excess energy is positive.

From this result, we can consider that in **Pd-Pt alloy particles, a structure in which Pd is in the outer shell is likely to be stable in nature**.

Finally, let us find the excess energy of a structure with 306 Pt and 405 Pd randomly arranged.

In [13]:
Pd405Pt306_random = Pt711.copy()
# randomly choose 405 atoms to be replaced to Pd
pd_indices = np.random.choice(np.arange(711), 405, replace=False)
Pd405Pt306_random.numbers[pd_indices] = atomic_numbers["Pd"]

print(f"Replaced to Pd with indices {pd_indices}")
view_x3d(Pd405Pt306_random)

Replaced to Pd with indices [322 302 324 482 122 194 640 292 544 616 388 107  76 326 554 243 629 372
 476 430 691  51 518  59 678 214 291 273 660 172 531 581 174   3 488 510
 434 565 383 489 584 321  42 445 400  75 631 385 105 664  96 175 475 667
 366 386 423 357 285 682 153 221 465 265 295 155  62 504 677 588   9 185
  31 191 306  92  35 648  57 470   8 525  78 641  34 681 258   1 100 294
 610 524 440 555 198 701 241 353 200  45 237 159 707 206 567 199 205  55
  80  13 499 444  56 348  93 692 288 231 120 500  60  20 670 459 180 233
 213 121 182  71 332 439 254 627 653 345 363 253  19 367 522 225  77 509
 307 577 133 649 495 686 268 384 684 158 351 106 407 639 674 583 551  36
 703 412 319 139 690 232 598  67 582 552 520 304  43 305  74 491 485 650
 368 535 654 477   6 590 687 299 354 657 377 568 359 429  14 609 494 558
 501  86 625 267 277 560 358 592   2 538  39 167  83 579 356  99 523 316
 406 630 369 557 374 276 360 140 178 310 392 467 619 242  24  54 593  95
 573 441 343 426  18 13

In [14]:
print("Optimizing Pd405Pt306_random")
Pd405Pt306_random.calc = calculator
E_pd405pt306_random = get_opt_energy(Pd405Pt306_random)

Optimizing Pd405Pt306_random
      Step     Time          Energy          fmax
FIRE:    0 02:36:58      117.521908        0.605309
FIRE:    1 02:36:58      117.248950        0.546841
FIRE:    2 02:36:58      116.810640        0.440131
FIRE:    3 02:36:58      116.367031        0.328645
FIRE:    4 02:36:59      116.037383        0.380681
FIRE:    5 02:36:59      115.820314        0.407075
FIRE:    6 02:36:59      115.628656        0.339724
FIRE:    7 02:36:59      115.408214        0.294721
FIRE:    8 02:36:59      115.158416        0.199052
FIRE:    9 02:36:59      114.965903        0.247434
FIRE:   10 02:37:00      114.829190        0.291227
FIRE:   11 02:37:00      114.703896        0.220102
FIRE:   12 02:37:00      114.576295        0.173086
FIRE:   13 02:37:00      114.476686        0.185883
FIRE:   14 02:37:00      114.429803        0.181975
FIRE:   15 02:37:00      114.410956        0.154496
FIRE:   16 02:37:00      114.382987        0.105451
FIRE:   17 02:37:01      114.358935  

In [15]:
view_x3d(Pd405Pt306_random)

In [16]:
E_exess_pd405pt306_random = (E_pd405pt306_random - 405 / 711 * E_pd711 - 306 / 711 * E_pt711) / 711

print(f"Excess energy of Pd306Pt405 random = {E_exess_pd405pt306_random * 1000:.2f} meV")

Excess energy of Pd306Pt405 random = -4.37 meV


Again, negative excess energy is obtained, indicating that this structure can be stable.

In this case, the structure was created using a completely random configuration, but in reality, when substances are mixed, it may be possible to create a more stable structure by taking a localized pattern.
Monte Carlo methods are often used to optimize the placement patterns in this type of analysis, but it is omitted here. (A description of the Monte Carlo method and examples may be provided later in the future.)

Thus, by creating various structures and calculating their excess energy, it is possible to analyze what the stable structure of the alloy will be.

Reference

 - [Calculations of Real-System Nanoparticles Using Universal Neural Network Potential PFP](https://arxiv.org/abs/2107.00963)
 - [Structural Stability of Ruthenium Nanoparticles: A Density Functional Theory Study](https://pubs.acs.org/doi/10.1021/acs.jpcc.7b08672)
 - [Electronic Structure and Phase Stability of PdPt Nanoparticles](https://pubs.acs.org/doi/10.1021/acs.jpclett.5b02753)
 - [Electronic structure and phase stability of Pt3M (M = Co, Ni, and Cu) bimetallic nanoparticles](https://www.sciencedirect.com/science/article/abs/pii/S0927025620303657)
 - [The effect of SnO2(110) supports on the geometrical and electronic properties of platinum nanoparticles](https://link.springer.com/article/10.1007/s42452-019-1478-0)